# Configuration

Upon first import, `osyris` will create a configuration file located in `/home/user/.osyris/config_osyris.py`.
This will then be loaded by `osyris`, and allows easy configuration for units, constants and additional derived variables.

## Configure constants

This function creates additional constants or units that you use often, and do not wish to re-defined all the time.
The syntax is the following:

In [ ]:
def configure_constants(units):
    units.define('solar_mass = 1.9889e+33 * g = M_sun = M_sol')
    units.define('radiation_constant = 7.5659146e-015 * erg / cm^3 / K^4 = ar')

## Configure units

This function defines which units should be used for the different variables that are loaded when reading a Ramses output.
They are used to convert between code units and CGS.

Converting from code units requires the density, length and time scale factors `unit_d`, `unit_l` and `unit_t`,
found in the simulation `info` file.
Those are then used to define conversion factors for each variable, e.g. `density` or `position`.
The conversion factors are stored in a dictionary that is returned by the function (and subsequently stored in the `Dataset`).

An example snippet could look like this:

In [ ]:
def configure_units(units, unit_d, unit_l, unit_t):

    density = unit_d * units("g / cm**3")
    velocity = (unit_l / unit_t) * units("cm / s")
    momentum = density * velocity

    library = defaultdict(lambda: 1.0 * units('dimensionless'))

    library.update({
        'density': density,
        'velocity': velocity,
        'velocity_x': velocity,
        'velocity_y': velocity,
        'velocity_z': velocity,
        'momentum': momentum,
        'momentum_x': momentum,
        'momentum_y': momentum,
        'momentum_z': momentum})
    return library

## Additional variables

It is often useful to define some additional variables that are to be computed every time data is loaded.
These commonly include cell-centered magnetic field, or temperature.

It is recommended to place your variables in a `try/except` block,
which will prevent errors if the variables are not found,
for instance when loading data from a different simulation.

For instance

In [ ]:
def additional_variables(data):
    # Magnetic field
    try:
        data["hydro"]["B_field"] = 0.5 * (data["hydro"]["B_left"] + data["hydro"]["B_right"])
    except KeyError:
        pass

    # Mass
    try:
        data["hydro"]["mass"] = (data["hydro"]["density"] * data["amr"]["dx"]**3).to("M_sun")
    except KeyError:
        pass